<a href="https://colab.research.google.com/github/Tranvanhien141625/BDS/blob/main/Randomforest_HN-Grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount( '/content/gdrive' )


Mounted at /content/gdrive


In [ ]:
!pip install pyspark==3.0.1 py4j==0.10.9
8

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('train').getOrCreate()


In [ ]:
#data  =  spark.read.csv('/content/gdrive/MyDrive/VN_housing_dataset.csv', sep= ',',header=True ,inferSchema = True)


In [ ]:
#data.head(3)

In [ ]:
#data.printSchema()

In [ ]:
#data.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_file_path = "/content/gdrive/MyDrive/VN_housing_dataset.csv"
dataset_df = pd.read_csv(train_file_path)
print("Full train dataset shape is {}".format(dataset_df.shape))

In [ ]:
dataset_df

In [ ]:
dataset_df = dataset_df.drop('Unnamed: 0', axis=1)
dataset_df.head(100)



```
# Định dạng của đoạn này là mã`
```

Xóa ký tự toán học và chú thích sau số.

In [ ]:
#for index, value in dataset_df['Giá/m2'].items():
#    try:
#        converted_value = float(value)
#    except ValueError:
#        print(f"Lỗi chuyển đổi kiểu dữ liệu tại dòng {index}: {value}")


In [ ]:
dataset_df=dataset_df.drop(52317, axis=0, inplace=False)
dataset_df=dataset_df.drop(55653, axis=0, inplace=False)
dataset_df=dataset_df.drop(58003, axis=0, inplace=False)
dataset_df=dataset_df.drop(63, axis=0, inplace=False)



In [ ]:

dataset_df['Giá/m2'] = dataset_df['Giá/m2'].replace(to_replace=',', value='.', regex=True)
dataset_df['Giá/m2'] = dataset_df['Giá/m2'].replace('[^\d.]', '', regex=True).astype(float)

dataset_df['Dài'] = dataset_df['Dài'].replace('[^\d.]', '', regex=True)
dataset_df['Dài'] = dataset_df['Dài'].replace('[^\d.]', '', regex=True).astype(float)

dataset_df['Rộng'] = dataset_df['Rộng'].replace('[^\d.]', '', regex=True)
dataset_df['Rộng'] = dataset_df['Rộng'].replace('[^\d.]', '', regex=True).astype(float)

dataset_df['Số phòng ngủ'] = dataset_df['Số phòng ngủ'].replace('[^\d.]', '', regex=True)
dataset_df['Số phòng ngủ'] = dataset_df['Số phòng ngủ'].replace('[^\d.]', '', regex=True).astype(float)

dataset_df['Diện tích'] = dataset_df['Diện tích'].replace('[^\d.]', '', regex=True)
dataset_df['Diện tích'] = dataset_df['Diện tích'].replace('[^\d.]', '', regex=True).astype(float)



dataset_df = dataset_df.rename(columns={'Địa chỉ': 'Dia_chi'})
dataset_df = dataset_df.rename(columns={'Loại hình nhà ở': 'Loai_nha'})
dataset_df = dataset_df.rename(columns={'Giấy tờ pháp lý': 'Giay_to'})
dataset_df = dataset_df.rename(columns={'Số tầng': 'So_tang'})
dataset_df = dataset_df.rename(columns={'Số phòng ngủ': 'So_phong'})
dataset_df = dataset_df.rename(columns={'Diện tích': 'Dien_tich'})





In [ ]:
dataset_df=dataset_df.dropna()

In [ ]:
dataset_df.head()

In [ ]:
dataset_df.describe()


In [ ]:
dataset_df.head(10)


In [ ]:
print(dataset_df['Giá/m2'].describe())
plt.figure(figsize=(9, 8))
sns.distplot(dataset_df['Giá/m2'], color='g', bins=10, hist_kws={'alpha': 0.4});

In [ ]:
list(set(dataset_df.dtypes.tolist()))

In [ ]:
df_num = dataset_df.select_dtypes(include = ['float64', 'int64'])
df_num.head()

In [ ]:
df_num.hist(figsize=(16, 20), bins=10, xlabelsize=8, ylabelsize=8);

In [ ]:
y = dataset_df['Giá/m2']
X = dataset_df.drop(["Giá/m2"], axis=1)
X = dataset_df.drop(["Ngày"], axis=1)

In [ ]:


from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,train_size=0.8, random_state=0)

In [ ]:
# categorical_cols=[cname for cname in X_train.columns
#                   if X_train[cname].nunique()<10 and X_train[cname].dtype == "object"]

In [ ]:
# numerical_cols=[cname for cname in X_train.columns
#                  if X_train[cname].dtype in ["int64", "float64"]]


In [ ]:
X_train.head()

In [ ]:
numerical_cols=['Dien_tich','Dài','Rộng']
categorical_cols=['Dia_chi','Quận','Huyện','Loai_nha','Giay_to',]

In [ ]:
print("The number of categorical columns: ", len(categorical_cols))
print("The number of numerical columns: ", len(numerical_cols))

In [ ]:
my_cols=categorical_cols+numerical_cols
X_train = X_train[my_cols]
X_val = X_val[my_cols]


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
numerical_transformer = Pipeline(steps=[
    ("imputer_num", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

In [ ]:
categorical_transformer = Pipeline(steps = [
    ("imputer_cal", SimpleImputer(strategy="most_frequent")),
    ("onehot", LabelEncoder(handle_unknown="ignore"))
])

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ("num", numerical_transformer, numerical_cols),
    ("cat", categorical_transformer, categorical_cols)
])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)

In [ ]:
my_pipeline = Pipeline(steps=[ ("preprocessor", preprocessor),("model", rf)])

In [ ]:
my_pipeline.fit(X_train, y_train)

In [ ]:
val_preds = my_pipeline.predict(X_val)

In [ ]:
from sklearn.metrics import mean_absolute_error
print("Validation MAE: ", mean_absolute_error(y_val, val_preds))

In [ ]:
from sklearn.model_selection import cross_val_score
scores = -1 * cross_val_score(my_pipeline, X,y, cv = 5, scoring="neg_mean_absolute_error")
print("Mean Cross Validation Score: ", scores.mean())

Mean Cross Validation Score:  29.600208889042655


Tạo lưới

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'model__n_estimators': [250],
    'model__max_features': ['auto','sqrt','log2'],
    'model__max_depth' : [5],
    'model__criterion' :['absolute_error']}
GridCV = GridSearchCV(my_pipeline, param_grid, n_jobs= -1)
GridCV.fit(X_train,y_train)
print(GridCV.best_params_)
print(GridCV.best_score_)

Tạo lưới kết hợp các cách chuẩn hóa khác nhau.

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
# from sklearn.preprocessing import MinMaxScaler, RobustScaler, Normalizer
# from sklearn.preprocessing import OneHotEncoder


# #strategies_categorical = ['labelencoder', 'ordinalencoder', 'binaryencoder','onehotencoder']
# #strategies_numerical = ['minmaxscaler', 'robustscaler', 'normalizer']

# strategies_categorical = ['OneHotEncoder']
# strategies_numerical = [ 'normalizer','robustscaler']

# best_params_score=[]


# # Tạo bước tiền xử lý cho dữ liệu phân loại và số
# for strategy_cate in strategies_categorical:
#   for strategy_numer in strategies_numerical:
#     if (strategy_cate == 'OneHotEncoder')&(strategy_numer=='minmaxscaler'):
#       categorical_transformer = Pipeline(steps = [
#           ("imputer_cal", SimpleImputer(strategy="most_frequent")),
#           ("onehot", OneHotEncoder(handle_unknown="ignore"))
#       ])

#       numerical_transformer = Pipeline(steps=[
#             ("imputer_num", SimpleImputer(strategy="median")),
#             ("scaler", MinMaxScaler())
#         ])



#     if (strategy_cate == 'OneHotEncoder')&(strategy_numer=='normalizer'):
#       categorical_transformer = Pipeline(steps = [
#           ("imputer_cal", SimpleImputer(strategy="most_frequent")),
#           ("onehot", OneHotEncoder(handle_unknown="ignore"))
#       ])
#       numerical_transformer= Pipeline(steps=[
#             ("imputer_num", SimpleImputer(strategy="median")),
#             ("scaler", Normalizer())
#         ])
#     preprocessor = ColumnTransformer(transformers=[
#       ("num", numerical_transformer, numerical_cols),
#       ("cat", categorical_transformer, categorical_cols)
#         ])
#     my_pipeline = Pipeline(steps=[ ("preprocessor", preprocessor),("model", rf)])


#     param_grid = {
#         'model__n_estimators': [250],
#         'model__max_features': ['sqrt'],
#         'model__max_depth' : [4,5],
#         'model__criterion' :['absolute_error']}
#     GridCV = GridSearchCV(my_pipeline, param_grid, n_jobs= -1)
#     GridCV.fit(X_train,y_train)

#     best_params_score.append([str(GridCV.best_params_),GridCV.best_score_])







In [ ]:
 best_params_score